<a href="https://colab.research.google.com/github/Charmiece/Final-Project---CreditCard-Fraud/blob/Afreensiddiqa/Copy_of_ETL_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.1'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/graphics-driver

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-06 21:45:15--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  1.22MB/s    in 0.8s    

2022-04-06 21:45:17 (1.22 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://fraudcreditcard.s3.amazonaws.com/clean_data_train.csv"
spark.sparkContext.addFile(url)
user_data_df = spark.read.csv(SparkFiles.get("clean_data_train.csv"), sep=",", header=True, inferSchema=True)

In [5]:
# Show DataFrame
user_data_df.show()

+----------+--------------------+-------------+------+------+----------+---------+------------------+--------+---+
|Unnamed: 0|            merchant|     category|   amt|gender| unix_time|merch_lat|        merch_long|is_fraud|age|
+----------+--------------------+-------------+------+------+----------+---------+------------------+--------+---+
|         0|fraud_Rippin, Kub...|     misc_net|  4.97|     F|1325376018|36.011293|        -82.048315|       0| 34|
|         1|fraud_Heller, Gut...|  grocery_pos|107.23|     F|1325376044|49.159047|       -118.186462|       0| 44|
|         2|fraud_Lind-Buckridge|entertainment|220.11|     M|1325376051|43.150704|       -112.154481|       0| 60|
|         3|fraud_Kutch, Herm...|gas_transport|  45.0|     M|1325376076|47.034331|       -112.561071|       0| 55|
|         4| fraud_Keeling-Crist|     misc_pos| 41.96|     M|1325376186|38.674999|        -78.632459|       0| 36|
|         5|fraud_Stroman, Hu...|gas_transport| 94.63|     F|1325376248|40.65338

In [6]:
age_fraud = user_data_df[['age', 'is_fraud']]
age_fraud.show()

+---+--------+
|age|is_fraud|
+---+--------+
| 34|       0|
| 44|       0|
| 60|       0|
| 55|       0|
| 36|       0|
| 61|       0|
| 29|       0|
| 75|       0|
| 81|       0|
| 48|       0|
| 32|       0|
| 56|       0|
| 33|       0|
| 77|       0|
| 55|       0|
| 57|       0|
| 70|       0|
| 84|       0|
| 76|       0|
| 42|       0|
+---+--------+
only showing top 20 rows



In [7]:
gender_fraud = user_data_df[['gender', 'is_fraud']]
gender_fraud.show()

+------+--------+
|gender|is_fraud|
+------+--------+
|     F|       0|
|     F|       0|
|     M|       0|
|     M|       0|
|     M|       0|
|     F|       0|
|     F|       0|
|     M|       0|
|     F|       0|
|     F|       0|
|     M|       0|
|     F|       0|
|     M|       0|
|     M|       0|
|     M|       0|
|     M|       0|
|     F|       0|
|     M|       0|
|     M|       0|
|     M|       0|
+------+--------+
only showing top 20 rows



In [8]:
location_info = user_data_df[['is_fraud', 'merchant', 'merch_lat', 'merch_long']]
location_info.show()

+--------+--------------------+---------+------------------+
|is_fraud|            merchant|merch_lat|        merch_long|
+--------+--------------------+---------+------------------+
|       0|fraud_Rippin, Kub...|36.011293|        -82.048315|
|       0|fraud_Heller, Gut...|49.159047|       -118.186462|
|       0|fraud_Lind-Buckridge|43.150704|       -112.154481|
|       0|fraud_Kutch, Herm...|47.034331|       -112.561071|
|       0| fraud_Keeling-Crist|38.674999|        -78.632459|
|       0|fraud_Stroman, Hu...|40.653382|-76.15266700000001|
|       0|fraud_Rowe-Vander...|37.162705|        -100.15337|
|       0|fraud_Corwin-Collins|38.948089|        -78.540296|
|       0|    fraud_Herzog Ltd|40.351813|        -79.958146|
|       0|fraud_Schoen, Kup...|37.179198|        -87.485381|
|       0|fraud_Rutherford-...|40.275891|        -96.011548|
|       0|fraud_Kerluke-Abs...|40.103866|        -78.624459|
|       0|   fraud_Lockman Ltd|27.630593|        -82.308891|
|       0|     fraud_Kie

In [9]:
fraud_df = user_data_df[['is_fraud', 'category', 'amt']]
fraud_df.show()

+--------+-------------+------+
|is_fraud|     category|   amt|
+--------+-------------+------+
|       0|     misc_net|  4.97|
|       0|  grocery_pos|107.23|
|       0|entertainment|220.11|
|       0|gas_transport|  45.0|
|       0|     misc_pos| 41.96|
|       0|gas_transport| 94.63|
|       0|  grocery_net| 44.54|
|       0|gas_transport| 71.65|
|       0|     misc_pos|  4.27|
|       0|  grocery_pos|198.39|
|       0|  grocery_pos| 24.74|
|       0| shopping_net|  7.77|
|       0|  grocery_pos| 71.22|
|       0|  grocery_pos| 96.29|
|       0| shopping_pos|  7.77|
|       0| shopping_net|  3.26|
|       0|     misc_net| 327.0|
|       0| shopping_pos|341.67|
|       0|  food_dining| 63.07|
|       0|  grocery_pos| 44.71|
+--------+-------------+------+
only showing top 20 rows



Connect to the AWS RDS instance and write each DataFrame to its table.

In [10]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://finalproject.cylfu7wfg2zs.us-east-1.rds.amazonaws.com:5432/final_DB"
config = {"user":"postgres", 
          "password": "fraudcc1", 
          "driver":"org.postgresql.Driver"}

In [11]:
# Write age_fraud to table in RDS
age_fraud.write.jdbc(url=jdbc_url, table='age_fraud', mode=mode, properties=config)

In [12]:
# Write gender_fraud to table in RDS
gender_fraud.write.jdbc(url=jdbc_url, table='gender_fraud', mode=mode, properties=config)

In [13]:
# Write location_info to table in RDS
location_info.write.jdbc(url=jdbc_url, table='location_info', mode=mode, properties=config)

In [14]:
# Write fraud_df to table in RDS
fraud_df.write.jdbc(url=jdbc_url, table='fraud_df', mode=mode, properties=config)